In [1]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMapWithTime
from folium.plugins import TimestampedGeoJson
from numpy import nan
import math
import datetime


import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import seaborn as sns

In [85]:
inno_geocoded = pd.read_csv("inno_geocoded.csv")
gdf = gpd.GeoDataFrame(inno_geocoded, geometry=gpd.points_from_xy(inno_geocoded.lng, inno_geocoded.lat))

c:\Users\eric9\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (53) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [86]:
inno2 = gdf["inno_nr"] == "inno_34"
inno2 = gdf[inno2]
inno2 = inno2[inno2["lat"].notna()]
inno2["created_at_td"] = pd.to_datetime(inno2["created_at"])

In [90]:
inno2["created_at_int"] = inno2["created_at_td"].apply(lambda x:x.toordinal())

In [91]:
norm = matplotlib.colors.Normalize(vmin= 735272,vmax= 737397, clip=True)
mapper = plt.cm.ScalarMappable(norm=norm, cmap=plt.cm.viridis)
inno2["color"] = inno2["created_at_int"].apply(lambda x: mcolors.to_hex(mapper.to_rgba(x)))

In [92]:

df = inno2.copy()
x_coords = df["geometry"].x.to_list()
y_coords = df["geometry"].y.to_list()
coordinate_list = [[x, y] for x, y in zip(x_coords, y_coords)]
df["created_at_td"] = pd.to_datetime(df["created_at_td"])
times_list = [i.isoformat() for i in df["created_at_td"].to_list()]
colorlist = df["color"].to_list()

feature_list = [
    {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": coordinate_list,
        },
        "properties": {
            "times": times_list,
            "style": {"color": "blue", "weight": 1},
            "icon": "circle",
            "iconstyle": {"fillColor": "#ff0000", "fillOpacity": 1, "stroke": "true", "radius": 5}
        },
    }
]

In [93]:
map = folium.Map([47.32, 8.50], zoom_start=2, tiles = "CartoDB dark_matter", control_scale = True)

TimestampedGeoJson(
    feature_list,
    period = 'PT1H',
    #duration = 'PT1M',
    transition_time = 1,
    auto_play = True).add_to(map)

map